In [ ]:
# рекурсивную реализацию всех перестановок пока отложила 

## Задача A. Все перестановки заданной длины

In [24]:
from itertools import permutations

n = int(input())
for i in permutations(range(1,n+1)):
    perm = i
    new_perm = [str(x) for x in perm]
    print(''.join(new_perm))

2
12
21


## Задача B. Затерянный мир

Расстановка N ферзей на поле размером N*N, чтобы ферзи не били друг друга 

(это нельзя по 1 строке, по 1 столбцу и по 2 диагоналям)

In [67]:
def ferz(col, n, diag, visited_row, visited_col, rez):
    if col == (n+1):
        return 
    for row in range(1, n+1):
        if visited_row[row] == 0: # эту строку мж посетить (она свободна)
            # надо проверить доступность диагоналей 
            for elem in diag:
                if len(elem) != 0: # по пустым не идем 
                    # совпадение уже хотя бы по 1 - нельзя 
                    if (elem[0] + elem[1]) == (row + col):
                        break
                    elif (elem[0] - elem[1]) == (row - col):
                        break
            else: # если по всем диагоналям прошли проверку 
                visited_row[row] = 1
                visited_col[col] = row 
                #diag.append((row, col))
                diag[col] = [row, col]
                ferz(col+1, n, diag, visited_row, visited_col, rez)
    if col == n and sum(visited_row) == n: # посетили все строки
        rez.append(1)
    if col != 1: # только не для 1 колонки
        numb = visited_col[col] # занятая строка этой колонки 
        visited_row[numb] = 0 # строка стала свободной
        visited_col[col] = 0 # не посетили никакую строку 
        diag[col] = []
        numb = visited_col[col-1] # занятая строка этой колонки 
        visited_row[numb] = 0 # строка стала свободной
        visited_col[col-1] = 0 # не посетили никакую строку 
        diag[col-1] = []
        
n = int(input())  
visited_row = [0] * (n+1)
visited_col = [0] * (n+1)
diag = [[] for i in range(n+1)]
rez = []

ferz(1, n, diag, visited_row, visited_col, rez)
print(len(rez))

# РАБОЧЕЕ РЕШЕНИЕ (с 1 раза!)

8
92


In [61]:
sp = [1,1,1,1,1]
sum(sp)

5

## Задача D. Простая задача коммивояжера

Неориентированный взвешенный граф задан матрицей смежности. Найдите кратчайший цикл, который начинается и заканчивается в вершине номер 1 и проходит через все вершины по одному разу. 

https://contest.yandex.ru/contest/53032/problems/D/

In [35]:
# обход графа в глубину 
def dfs(graph, visited, now):
    visited[now] = True 
    for neig in graph[now]: 
        if not visited[neig]:
            dfs(graph, visited, neig)

In [49]:
import heapq

# нам нужна куча минимумов 
sp = [100000]
heapq.heapify(sp)
heapq.heappush(sp, 12)
heapq.heappush(sp, 11)
heapq.heappush(sp, 100)
heapq.heappush(sp, 132)
heapq.heappush(sp, 12)
sp[0]

11

In [50]:
sp = [0,0,1,2,3,0]
sp.count(0)

3

In [58]:
# РАБОЧЕЕ РЕШЕНИЕ 

import sys
sys.setrecursionlimit(100001)
import heapq

def dfs(graph, visited, now, ancestor, rez, f):
    visited[now] = True
    # print('tmp vertex', now)
    colors[now] = 1
    predok[now] = ancestor
    ancestor = now
    for neig in graph[now]:
        if not visited[neig]:
            dfs(graph, visited, neig, ancestor, rez, f)
    colors[now] = 2
    for neig in graph[now]:
        if colors[neig] == 1 and neig != predok[now]:
            # print('YES')
            f = True
            predok[neig] = now
            break
    if f:
        rez.append(now)
        elem = predok[now]
        while elem != now:
            rez.append(elem)
            elem = predok[elem]
        # print('loops elements', rez)
        raise Exception


def commi_voyage(M_gr, n, visited, prev, temp_vert, dist, tmp_min):
    visited[temp_vert] = True
    # print(temp_vert)
    neighs = M_gr[temp_vert - 1]
    f = True
    for i in range(n):
        # (i+1) - номер соседа
        # если ребро существует и не посещали этого соседа
        if neighs[i] != 0 and (not visited[i + 1]):
            prev[i + 1] = temp_vert
            # дистанция до родителя + ребро между ними
            dist[i + 1] = dist[temp_vert] + neighs[i]
            # if dist[i+1]  tmp_min[0]:
            f = False
            commi_voyage(M_gr, n, visited, prev, i + 1, dist, tmp_min)
    if f:  # только в этом случае будем обновлять distance (дошли до конца ветки - больше нет соседей)
        heapq.heappush(tmp_min, dist[temp_vert] + M_gr[temp_vert - 1][0])
    visited[temp_vert] = False
    # print('dist', dist[temp_vert])
    dist[temp_vert] = 0


n = int(input())

M_gr = [[] for i in range(n)]
for i in range(n):
    sp = list(map(int, input().split()))
    M_gr[i] = sp
if n == 1:
    print(0)
elif n == 2:
    if M_gr[0][1] == 0:
        print(-1)
    else:
        visited = [False] * (n + 1)
        prev = [None] * (n + 1)
        dist = [0] * (n + 1)
        tmp_min = [100000000]
        heapq.heapify(tmp_min)  # теперь эта структура куча
        commi_voyage(M_gr, n, visited, prev, 1, dist, tmp_min)
        print(tmp_min[0])
else:
    # надо превратить из матрицы смежности в список смежности
    graph = [[] for i in range(n + 1)]
    for i in range(n):
        for j in range(n):
            if M_gr[i][j] != 0:
                graph[i + 1].append(j + 1)
    #print(graph)
    rez = []
    visited = [False] * (n + 1)
    visited[0] = True
    colors = [0] * (n + 1)
    predok = [0] * (n + 1)
    ancestor = 0
    logic = True  # нет цикла
    f = False
    try:
        dfs(graph, visited, 1, ancestor, rez, f)
    except Exception:  # есть цикл
        logic = False  # есть цикл
        visited = [False] * (n + 1)
        prev = [None] * (n + 1)
        dist = [0] * (n + 1)
        tmp_min = [100000000]
        heapq.heapify(tmp_min)  # теперь эта структура куча
        commi_voyage(M_gr, n, visited, prev, 1, dist, tmp_min)
        print(tmp_min[0])
    if logic:  # цикла в графе нет
        print(-1)

1
0
0


## Задача С. Максимальный срез 

Взвешенный неориентированный граф без петель задан матрицей смежности. Распределите вершины по двум долям так, чтобы сумма весов рёбер, соединяющих вершины из разных долей, была максимальна.

(0 <= N <= 20 - кол-во вершин) 

In [87]:
n = int(input())
M_gr = [[] for i in range(n)]
for i in range(n):
    M_gr[i] = list(map(int, input().split()))
    
mx = 0
rez = []
for k in range(1, 2**n-1):
    sp = [int(i) for i in bin(k)[2:].zfill(n)]
    #print(*sp)
    if sp[0] == 0: # первая вершина всегда в 0 
        tmp = 0
        for i in range(n):
            if sp[i] == 0: # ищем путь только от 0 до 1
                for j in range(n):
                    if sp[j] == 1:
                        tmp += M_gr[i][j]
                #print(tmp)
        if tmp > mx:
            mx = tmp
            rez = sp
                
print(mx)
sp = [i+1 for i in sp]
print(*sp)

4
0 10 3 0
10 0 7 2
3 7 0 9
0 2 9 0
0 0 0 1
0
2
11
0 0 1 0
3
10
19
0 0 1 1
3
12
0 1 0 0
10
17
19
0 1 0 1
10
26
0 1 1 0
13
24
0 1 1 1
13
1 0 0 0
10
13
13
1 0 0 1
12
24
1 0 1 0
17
26
1 0 1 1
19
1 1 0 0
10
12
1 1 0 1
19
1 1 1 0
11
26
[1, 1, 1, 0]


In [78]:
[int(i) for i in bin(3)[2:].zfill(3)]

[0, 1, 1]

In [89]:
n = 1
a = []
 
while n:
    a.append(n % 2)
    n //= 2
    
a.reverse()
print(a)

[1]


## Задача E. Генерация правильных скобочных последовательностей

In [1]:
def is_good_bracket(rez):
    stack = []
    for skobk in rez:
        if skobk == '(' or skobk == '[' or skobk == '{':
            stack.append(skobk)
        else: # встретили закрыв скобку
            if len(stack) == 0:
                return False
            else: # длина стэка не 0
                last_elem = stack[-1] # последний элемент
                if (skobk == '}' and last_elem == '{') or (skobk == ')' and last_elem == '(') or (skobk == ']' and last_elem == '['):
                    stack.pop()
                else:
                    return False
    if len(stack) == 0:
        return True
    return False


def good_brackets_adv(n, brackets, rez):
    # в стеке нет закрыв скобки
    if (len(rez) != 0 and rez[0] in (']', ')')):
        rez.pop()
        return
    if (len(rez) == n):
        if is_good_bracket(rez):
            print(''.join(rez))
        rez.pop()
        return
    for elem in brackets:
        rez.append(elem)
        good_brackets_adv(n, brackets, rez)
    if (len(rez) != 0):
        rez.pop()

n = int(input())
if (n % 2 == 0 and n != 0):
    brackets = ['(', '[', ')', ']']
    rez = []
    good_brackets_adv(n, brackets, rez)
    
# Рабочий код в лоб, НО тайм лимит 

6
((()))
(([]))
(()())
(()[])
(())()
(())[]
([()])
([[]])
([]())
([][])
([])()
([])[]
()(())
()([])
()()()
()()[]
()[()]
()[[]]
()[]()
()[][]
[(())]
[([])]
[()()]
[()[]]
[()]()
[()][]
[[()]]
[[[]]]
[[]()]
[[][]]
[[]]()
[[]][]
[](())
[]([])
[]()()
[]()[]
[][()]
[][[]]
[][]()
[][][]


In [2]:
def good_brackets_adv(stack, max_len, brackets, rez):
    if (len(stack) > max_len[-1] // 2):
        stack.pop()
        rez.pop()
        max_len.pop()
        return
        # в стеке нет закрыв скобки
    for elem in brackets:
        if elem == '(':  # открыв скобки в стэк добавляем
            stack.append('(')
            rez.append('(')
            max_len.append(max_len[-1])
            good_brackets_adv(stack, max_len, brackets, rez)
        elif elem == '[':  # открыв скобки в стэк добавляем
            stack.append('[')
            rez.append('[')
            max_len.append(max_len[-1])
            good_brackets_adv(stack, max_len, brackets, rez)
        else:  # встретили закрыв скобку
            # если закрыв скобка неправильного типа - ничего не делаем
            if (len(stack) != 0):  # только в случае непустого стека
                tmp_elem = stack[-1]
                if tmp_elem == '(' and elem == ')':
                    stack.pop()
                    rez.append(elem)
                    # последнее значение - 2
                    l = max_len[-1]
                    max_len.append(l-2)
                    good_brackets_adv(stack, max_len, brackets, rez)
                elif tmp_elem == '[' and elem == ']':
                    stack.pop()
                    rez.append(elem)
                    l = max_len[-1]
                    max_len.append(l - 2)
                    good_brackets_adv(stack, max_len, brackets, rez)

    if (len(rez) == n):
        print(''.join(rez))
        el = rez[-1]
        if el == ')':
            stack.append('(')
        elif el == ']':
            stack.append('[')
        rez.pop()
    elif (len(rez) != 0):
        if (len(stack) != 0 and rez[-1] == '(' or rez[-1] == '['):
            stack.pop()
        elif rez[-1] == ')':
            stack.append('(')
        elif rez[-1] == ']':
            stack.append('[')
        rez.pop()
    max_len.pop()


n = int(input())
if (n % 2 == 0 and n != 0):
    brackets = ['(', '[', ')', ']']  # все виды скобок в лексикографич порядке
    rez = []
    stack = []
    max_len = [n]
    good_brackets_adv(stack, max_len, brackets, rez)
    
# РАБОЧЕЕ РЕШЕНИЕ ! 

4
(())
([])
()()
()[]
[()]
[[]]
[]()
[][]
